In [52]:
import numpy as np
import requests
import re
import pandas as pd
import datetime
from functools import reduce
import random
import time
from random import randint

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1500

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [55]:
pd.read_csv('lockwood_notes.csv')

,id,case_name,related_case,year,maj_opinion,maj_joined,dissent_opinion,dissent_joined,importance,facts,issues,arguments,ruling,discussion,concur_opinion,concur_joined,concur2_opinion,concur2_joined,concur3_opinion,concur3_joined,dissent2_opinion,dissent2_joined,dissent3_opinion,dissent3_joined,justices_np,topic,sub_topic,law_class,law_classes
0,141,Barenblatt v. U.S.,NaN,1959,93.0,"90,81,96,95",79,"82,92",NaN,* House and American Activities Committee: Ind...,NaN,NaN,* Barenblatt: Congressional committees have a ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.0,NaN,NaN,NaN,NaN,First Amendment,NaN,American Constitution,American Constitution
1,140,Scott v. Emerson,"139,138,2",1852,NaN,NaN,NaN,NaN,"Dred Scott, Emerson’s slave, was taken out of ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Slavery,NaN,American Constitution,American Constitution
2,139,Strader v. Graham,"140,138,2",1851,NaN,NaN,NaN,NaN,State in which the suit is tried can apply its...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Slavery,NaN,American Constitution,American Constitution
3,138,Rachel v. Walker,"139,140,2",1836,NaN,NaN,NaN,NaN,"Slave from Missouri, resided in Wisconsin, and...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Slavery,NaN,American Constitution,American Constitution
4,137,Rose v. Council for Better Education,NaN,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Law and Society,NaN
5,136,Abbott v. Burke,NaN,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Law and Society,NaN
6,135,San Antonio School District v. Rodriguez,NaN,1973,103.0,"102,101,104,96",82,"100,97",* This basically killed the prospects of decad...,* San Antonio Independent School District was ...,"1. Did the federal district court, in applying...",NaN,"1. No, because education isn’t a fundamental r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Landmark Cases,"Tiers of Scrutiny,Equal Protection",Law and Society,"Law and Society,American Constitution"
7,134,Shelby County v. Holder,NaN,2013,114.0,"115,109,108,111",112,"113,117,116",* Gutted one of the most crucial portions of t...,* VRA 1965 enacted in response to a century of...,* In renewing Section 5 (under the constraints...,NaN,"* Yes. Section 4(b) is unconstitutional, as it...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voting Rights,"10th Amendment,Voting Rights",Law and Society,Law and Society
8,133,Shaw v. Reno,NaN,1993,106.0,"107,108,111",97,"102,110,105",NaN,"* During pre-clearance of North Carolina, the ...",* Did this reapportionment violent the 14th Am...,NaN,"* Yes. ""The State had created an unconstitutio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voting Rights,"Fourteenth Amendment,District Apportionment,10...",Law and Society,Law and Society
9,132,Baker v. Carr,130,1962,94.0,"79,90,82,96,92",93,"81,95",* Political Questions Doctrine,* Apportionment cases had often been brought u...,* Can the SCOTUS hear a malapportionment claim...,NaN,"* Yes. Under the Equal Protection clause, the ...",* Cases need to be brought under Equal Protect...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Judicial Review,Landmark Cases,Political Quest...","Equal Protection,Political Questions Doctrine,...",Law and Society,"Law and Society,American Constitution"


In [5]:
loc = pd.read_csv('../out/loc_info.csv').drop(columns=['Unnamed: 0'])

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
loc.loc[loc['doc_name'].str.contains('United States')]

NameError: name 'loc' is not defined

# Oyez data cleaning

In [363]:
oyez = pd.read_csv('../out/oyez_page_data.csv').drop(columns=['Unnamed: 0'])

oyez.loc[oyez['oyez_link']=='https://www.oyez.org/cases/1975/73-7031', 'revoked_order_of_june_23_1975'] = 'Feb 23, 1976'
# oyez

In [364]:
def extract_decisions(x):
    text = str(x)[2:-3].replace(', Jr', ',_Jr').replace("\'", "'")
    decisions_list = text.split(']], [')
    decision_dict = {}
    for num, text in enumerate(decisions_list):
        decision = {}
        if '], [' in text:
            dissent = text.split('], [')[-1][1:-1].split("', '")
            if len(str(dissent[0])) > 4:
                decision['dissent'] = dissent
            text = text.split('], [')[0]
        if ', [' in text:
            decision['majority'] = text.split(', [')[-1][1:-1].split("', '")
            text = text.split(', [')[0]
        if "', '" in text:
            info = text[1:-1].replace('\\', '').split("', '")
            if len(info) > 0:
                decision['decision'] = info[0]
            if len(info) > 1:
                decision['winner'] = info[1]
            if len(info) > 2:
                if info[2] != '':
                    decision['per_curiam'] = info[2]
            if len(info) > 3:
                decision['held'] = info[3]
        if decision != {}:
            decision_dict['decision ' + str(num + 1)] = decision
    if decision_dict != {}:
        return decision_dict

oyez['decision_dict'] = oyez['decisions'].apply(lambda x: extract_decisions(x))

In [365]:
def create_documents_dict(x):
    documents = {}
    document_list = []
    list_of_data_pairs = str(x)[4:-2].replace('  ', '').split("'], [")
    if list_of_data_pairs == ['']:
        return np.nan
    for data_pair in list_of_data_pairs:
        document = {}
        doc_info_list = data_pair.split("', '")
        document['type'] = doc_info_list[0]
        if ' (' in doc_info_list[0]:
            document['judge'] = doc_info_list[0].split(' (')[-1][:-2]
            document['type'] = doc_info_list[0].split(' (')[0].replace("' ", '')
        document['url'] = doc_info_list[1].replace("'", '')
        document_list.append(document)
    return document_list

oyez['document_list'] = oyez['documents'].apply(lambda x: create_documents_dict(x))
# oyez

In [366]:
oyez['citation_text'] = oyez['citation'].apply(lambda x: str(x).replace('\\xa0', ' ').replace('   ', '')[2:-2].split("', '")[0])
oyez['justia_link'] = oyez['citation'].apply(lambda x: str(x).replace('\\xa0', ' ').replace('   ', '')[2:-2].split("', '")[-1])

text_info_columns = ['facts', 'questions', 'conclusion']

for column in text_info_columns:
    oyez[column] = oyez[column].apply(lambda x: str(x).replace('\\xa0', ' ')[2:-2])
# oyez

In [362]:
# num = random.randint(0,100)
# # print(oyez.loc[num, 'oyez_link'])
# oyez.loc[num, 'timeline']
# # oyez.loc[0, 'timeline']

# {'syllabus': 'https://supreme.justia.com/cases/federal/us/561/08-1394/#tab-opinion-1963359',
#  'opinion of the court (ginsburg) ': 'https://supreme.justia.com/cases/federal/us/561/08-1394/#tab-opinion-1963360',
#  'concurring opinion (alito) ': 'https://supreme.justia.com/cases/federal/us/561/08-1394/#tab-opinion-1963357',
#  'concurring opinion (scalia) ': 'https://supreme.justia.com/cases/federal/us/561/08-1394/#tab-opinion-1963358',
#  'concurring in-part, dissenting in-part (sotomayor) ': 'https://supreme.justia.com/cases/federal/us/561/08-1394/#tab-opinion-1963356',
#  'view case': 'justia case text'}

In [361]:
# for column in oyez.columns:
#     print(column, len(oyez) - oyez[column].isna().sum())

In [374]:
timeline_columns = ['granted', 'dismissed', 'argued', 'decided', 'juris_postponed', 'affirmed_by_an_equally_divided_court', 'rehearing_granted', 'rehearing_granted,_judgment_vacated,_and_restored_to_the_calendar', 'rehearing_granted,_judgment_vacated_and_case_restored_for_reargument', 
                    'restored_to_the_calendar_and_set_for_reargument', 'revoked_order_of_june_23_1975', 'question_certified', 'question_certified_to_the_supreme_court_of_florida']
def isNaN(num):
    return num != num

def create_timeline_list(row):
    
    if ' (' in row['citation_text']:
        case_century = row['citation_text'].split(' (')[1][0:2]
    
    timeline_list = []
    for action_type in timeline_columns:
        dates = row[action_type]
        if isNaN(dates)==True:
            continue
        dates = str(dates).replace('   ', '')
        if ' - ' in dates:
            month = dates.split('  ')[0]
            year = dates.split(', ')[-1]
            days = dates.replace(month + '  ', '').replace(', ' + year, '').split(' - ')
            
            for num, day in enumerate(days):
                action = {}
                action['type'] = action_type
                action['date'] = pd.to_datetime(month + '/' + day + '/' + year)
                timeline_list.append(action)
        elif '; ' in dates:
            action_dates = dates.split('; ')
            for num, action_date in enumerate(action_dates):
                month = action_date.split('/')[0]
                day = action_date.split('/')[1]
                year = case_century + action_date.split('/')[-1]
                date = month + '/' + day + '/' + year
                action = {}
                action['type'] = action_type
                action['date'] = pd.to_datetime(date)
                timeline_list.append(action)
        elif 'pending' in dates.lower():
            action = {}
            action['type'] = action_type
            action['date'] = pd.to_datetime(month + '/' + day + '/' + year)
            timeline_list.append(action)
        else:
            action = {}
            month = dates.split(' ')[0]
            year = dates.split(', ')[1]
            day = dates.split(', ')[0].split(' ')[1]
            action['type'] = action_type
            action['date'] = pd.to_datetime(month + '/' + day + '/' + year)
            timeline_list.append(action)
    if timeline_list == []:
        return np.nan
    return timeline_list

oyez['timeline_list'] = oyez.apply(lambda x: create_timeline_list(x), axis=1)
# oyez

In [368]:
def create_advocates_dict(text):
    if isNaN(text)==True:
        return np.nan
    advocates = str(text)[2:-2].split('], [')
    advocate_list = []
    for advocate_text in advocates:
        advocate = {}
        advocate_text_list = advocate_text[1:-1].replace('"', "'").split("', '")
        advocate['name'] = advocate_text_list[0]
        advocate['oyez_link'] = advocate_text_list[-1]        
        if ', for ' in advocate_text_list[1]:
            advocate['position'] = advocate_text_list[1].split(', for ')[0]
            advocate['side'] = advocate_text_list[1].split(', for ')[-1]
        elif 'for ' in advocate_text_list[1]:
            advocate['side'] = advocate_text_list[1].split('for ')[-1]
        advocate_list.append(advocate)
    return advocate_list
    
oyez['advocate_list'] = oyez['advocates'].apply(lambda x: create_advocates_dict(x))
# oyez

In [369]:
def create_location_or_court_dict(x):
    if isNaN(x)==True:
        return np.nan
    text = str(x)[2:-1].replace('  ', '').replace('"', "'").replace(" ', '", "', '").split("', '")
    loc_or_court = {}
    loc_or_court['name'] = text[0]
    if 'pending' in loc_or_court.get('name').lower():
        return 
    loc_or_court['url'] = text[1]
    return loc_or_court

oyez['location_dict'] = oyez['location'].apply(lambda x: create_location_or_court_dict(x))
oyez['court_dict'] = oyez['decided_by'].apply(lambda x: create_location_or_court_dict(x))

In [370]:
def create_media_dict(x):
    if isNaN(x)==True:
        return np.nan
    media_list = []
    media_items = str(x)[3:-3].replace('"', "'").replace(" ', '", "', '").split('], [')
    for media_text in media_items:
        media = {}
        media_info_list = media_text[0:-1].replace("'Oral", 'Oral').split("', '")
        media['url'] = media_info_list[1]
        media_info = media_info_list[0]
        if ' (part' in media_info.lower():
            media['part'] = media_info.split(' (')[-1][0:-1].lower()
            media_info = media_info.split(' (')[0]
        if ' - ' in media_info:
            date = media_info.split(' - ')[-1].split(' (')[0]
            media['date'] = pd.to_datetime(date)
            media_info = media_info.split(date)[0].replace(' - ', '')
        if ', Part' in media_info:
            if 'part' in media:
                media['part+'] = 'part' + media_info.split(', Part')[-1]
            else:
                media['part'] = 'part' + media_info.split(', Part')[-1]
            media_info = media_info.split(', Part')[0]
        media['type'] = media_info
        media_list.append(media)
    return media_list

oyez['media_list'] = oyez['media'].apply(lambda x: create_media_dict(x))
# oyez

In [372]:
oyez['docket_no.'] = np.where(oyez['docket_no.'].str.contains('None')==True, np.nan, oyez['docket_no.'].str.replace(' ', ''))

In [376]:
oyez_columns = ['oyez_name', 'oyez_link', 'citation_text', 'docket_no.', 'court_dict', 'lower_court',
                'petitioner', 'respondent', 'appellant', 'appellee', 'facts', 'questions', 'conclusion', 
                'decision_dict', 'document_list', 'justia_link', 'timeline_list', 'advocate_list', 'location_dict', 'media_list']

oyez_info = oyez[oyez_columns]

In [377]:
oyez_info.to_csv('../out/oyez_info.csv')